# **Case QuantumFinance - Disciplina NLP - Classificador de chamados**

**Atenção:**
- Leia com atenção o descritivo do trabalho e as orientações do template.
- O trabalho deve ser entregue respeitando a estrutura do arquivo de template em notebook "Template_Trabalho_Final_NLP.ipynb" e compactado no formato .zip. Apenas um arquivo no formato .ipynb deve ser entregue consolidando todo o trabalho.

***Participantes (RM - NOME):***<br>
# RM 357053 - WILSON ROBERTO DE MELO
# RM 358310 - RAFAEL DE MIRANDA MEIRELLES COSTA E SILVA

### **Crie um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para alguma área especialista no assunto para uma melhor tratativa.​

1. Crie ao menos um modelo classificador de assuntos aplicando técnicas de NLP (PLN), Vetorização (n-grama + métrica) e modelo supervisionado, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do seu modelo.​

  O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

2. Utilizar ao menos uma aplicação de modelos com Embeddings usando Word2Vec e/ou LLM´s para criar o modelo classificador com os critérios do item 1. Não é necessário implementar aplicações usando serviços de API da OpenAI ou outros por exemplo.

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, aplicações de GenIA, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de 2 até 4 pessoas (mesmo grupo do Startup One) e trabalhos iguais serão descontado nota e passível de reprovação.

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

Bom desenvolvimento!

###**Area de desenvolvimento e validações**

Faça aqui as demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

---
---
---
TODO O DESENVOLVIMENTO FOI FEITO EM COLAB, CONECTADO EM L4
---
---
---

### Abaixo estão células, cópias do notebook "nlp_entrega.ipynb" usado para o desenvolvimento e comparação entre as metofologias, estando com os paths adequados àquilo que foi desenvolvido, tanto no GD e no SSD local.


In [ ]:
 🔧 ETAPA: MONTAGEM DO GOOGLE DRIVE E SALVAMENTO DO CSV NO DIRETÓRIO BASE

# 1️⃣ Monta o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2️⃣ Importa pandas
import pandas as pd

# 3️⃣ Carrega o CSV remoto com fallback para separador
url = 'https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv'

try:
    df = pd.read_csv(url, sep=None, engine='python')
except Exception:
    df = pd.read_csv(url, sep=';', engine='python')

# 4️⃣ Verifica estrutura
print(df.info())
print(df.head(20))

# 5️⃣ Salva no diretório mostrado na imagem
output_path = '/content/drive/MyDrive/MBA_NLP/bases_criadas/dados_originais.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f'\n✅ CSV salvo em: {output_path}')


In [ ]:
# 🔧 ETAPA: VERIFICAÇÃO E INSTALAÇÃO DE BIBLIOTECAS ESSENCIAIS

import sys

# Lista de bibliotecas obrigatórias
required_packages = [
    'pandas', 'numpy', 'scikit-learn',
    'nltk', 'spacy', 'unidecode',
    'tqdm', 'sentence-transformers'
]

# Flag para controlar se precisa rodar o download do modelo SpaCy
need_spacy = False

# Instala cada pacote se não estiver presente
for package in required_packages:
    try:
        __import__(package.replace('-', '_'))
        print(f'✅ {package} OK')
    except ImportError:
        print(f'⚙️ Instalando {package} ...')
        !{sys.executable} -m pip install {package}
        if package == 'spacy':
            need_spacy = True

# Download do modelo SpaCy pt_core_news_sm se necessário
if need_spacy:
    print("⚙️ Baixando modelo SpaCy pt_core_news_sm ...")
    !python -m spacy download pt_core_news_sm
else:
    print("✅ Verificando se modelo SpaCy já existe ...")
    try:
        import spacy
        spacy.load("pt_core_news_sm")
        print("✅ Modelo SpaCy pt_core_news_sm já está disponível.")
    except:
        print("⚙️ Baixando modelo SpaCy pt_core_news_sm ...")
        !python -m spacy download pt_core_news_sm

print("\n Verificação concluída.")


In [ ]:
# 🔧 ETAPA: DOWNLOAD E VALIDAÇÃO DO MODELO SPACY pt_core_news_sm

import spacy

try:
    nlp = spacy.load('pt_core_news_sm')
    print("✅ Modelo SpaCy `pt_core_news_sm` já está instalado e carregado.")
except OSError:
    print("⚙️ Baixando modelo SpaCy `pt_core_news_sm` ...")
    !python -m spacy download pt_core_news_sm
    nlp = spacy.load('pt_core_news_sm')
    print("✅ Download concluído e modelo carregado.")


In [ ]:
# 🔧 ETAPA: INSTALAÇÃO DO TOKENIZER NLTK E STOPWORDS

import nltk

# Download dos recursos necessários
nltk.download('punkt')
nltk.download('stopwords')

print("✅ Tokenizer `punkt` e stopwords em português prontos para uso.")


In [ ]:
# 🔧 ETAPA: DEFINIÇÃO DE CONSTANTES DE CAMINHO GLOBAIS

from pathlib import Path

# URL original do dataset (caso precise baixar novamente)
URL_ORIGINAL = 'https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv'

# Caminho base no Google Drive (ajuste conforme sua estrutura)
BASE_DIR = Path('/content/drive/MyDrive/MBA_NLP/bases_criadas')

# Caminho do arquivo CSV original salvo localmente
PATH_RAW = BASE_DIR / 'dados_originais.csv'

# Validação: o arquivo existe?
if PATH_RAW.exists():
    print(f'✅ Arquivo encontrado em: {PATH_RAW}')
else:
    print(f'⚠️ Atenção: {PATH_RAW} não encontrado! Verifique o salvamento.')

# Caminhos registrados
print(f"\n🌐 URL_ORIGINAL: {URL_ORIGINAL}\n📂 BASE_DIR: {BASE_DIR}")


In [ ]:
# 🔧 ETAPA: CARREGAMENTO DO DATASET LOCAL E VALIDAÇÃO EDA INICIAL

import pandas as pd

# Carrega o arquivo local validado
df = pd.read_csv(PATH_RAW)

# Informações gerais
print("=== Estrutura do DataFrame ===")
print(df.info())

print("\n=== Dimensão do DataFrame ===")
print(f"Shape: {df.shape}")

print("\n=== Primeiras 20 linhas ===")
print(df.head(20))


In [ ]:
# 🔧 ETAPA: ANÁLISE DE NULOS, HISTOGRAMA DE TEXTO E BALANCEAMENTO DE CLASSES

import matplotlib.pyplot as plt

# 1️⃣ Percentual de nulos por coluna
print("=== Percentual de valores nulos por coluna ===")
percent_nulos = df.isnull().mean() * 100
print(percent_nulos)

# 2️⃣ Histograma do tamanho dos textos de reclamação
df['text_length'] = df['descricao_reclamacao'].astype(str).apply(len)

plt.figure(figsize=(10,6))
plt.hist(df['text_length'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribuição do Tamanho dos Textos de Reclamação')
plt.xlabel('Número de Caracteres')
plt.ylabel('Frequência')
plt.show()

# 3️⃣ Contagem de classes na coluna-alvo 'categoria'
print("\n=== Contagem de amostras por categoria ===")
print(df['categoria'].value_counts())

# Gráfico de barras das classes
df['categoria'].value_counts().plot(kind='bar', figsize=(10,6), color='coral', edgecolor='black')
plt.title('Distribuição das Categorias de Reclamação')
plt.xlabel('Categoria')
plt.ylabel('Contagem')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 4️⃣ Verifica as primeiras linhas com o campo de tamanho de texto
print("\n=== Exemplo com comprimento de texto ===")
print(df[['descricao_reclamacao', 'text_length']].head(20))


=== Percentual de valores nulos por coluna ===
id_reclamacao           0.0
data_abertura           0.0
categoria               0.0
descricao_reclamacao    0.0
dtype: float64


=== Contagem de amostras por categoria ===
categoria
Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: count, dtype: int64


=== Exemplo com comprimento de texto ===
                                 descricao_reclamacao  text_length
0   Bom dia, meu nome é xxxx xxxx e agradeço se vo...          505
1   Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...          350
2   O cartão Chase foi relatado em xx/xx/2019. No ...          228
3   Em xx/xx/2018, enquanto tentava reservar um ti...         1577
4   Meu neto me dê cheque por {$ 1600,00} Eu depos...          607
5                        Você pode remover a consulta           28
6   Sem aviso prévio J.P. Morgan Chase restringiu ...         2409
7   Durante os meses de verão, experimento uma ren...         1493
8   Em xxxx xx/xx/2019, fiz um pagamento {$ 300.00...         3795
9   Eu tenho um cartão de crédito Chase que está r...          155
10     Mishandling desta conta por Chase Auto e XXXX.           46
11  Entrei em contato com o XXXX várias vezes na t...          696
12  Abri uma conta no Chase Bank no xxxx e usei um...          430
13  Para quem possa interessar, o Chase Bank cobro...          654
14  Meu cartão Chase Amazon foi recusado para uma ...         2205
15  Abri a conta de poupança para o bônus {$ 25,00...          582
16  Xxxx xxxx um sofá, assento de amor, mesa e cad...          186
17  Meu cartão desapareceu e eu não percebi até ho...          203
18  Chase me enviou um e -mail hoje com o título i...         1010
19  Fiz uma compra com xxxx xxxx xxxx em xx/xx/201...         2087


In [ ]:
# 🔧 ETAPA: FUNÇÃO DE LIMPEZA BÁSICA DE TEXTO (Etapa 2.1)

import re
from unidecode import unidecode

# Define função de limpeza
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = unidecode(text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove pontuação e números
    text = re.sub(r'\s+', ' ', text).strip()  # Remove espaços extras
    return text

# Aplica no campo original
df['texto_limpo'] = df['descricao_reclamacao'].apply(clean_text)

# Verifica amostra
print(df[['descricao_reclamacao', 'texto_limpo']].head(20))


In [ ]:
# 🔧 ETAPA: SUBSTITUIÇÃO DE PLACEHOLDERS E SALVAMENTO DO DATAFRAME

import re

def replace_xxxx_tokens(text):
    if pd.isnull(text):
        return ""

    # Substitui datas anonimizadas
    text = re.sub(r'\b(?:x{2}/x{2}/x{2,4})\b', '<DATE>', text)

    # Substitui nomes
    text = re.sub(r'\bnome\s+e?\s+x{2,}\b', '<PII>', text)
    text = re.sub(r'\bnome\s+x{2,}\b', '<PII>', text)

    # Substitui conta/cartão
    text = re.sub(r'\bconta\s+x{2,}\b', '<ID>', text)
    text = re.sub(r'\bcartao\s+x{2,}\b', '<ID>', text)

    # Substitui qualquer xxxx residual
    text = re.sub(r'\b[x]{2,}\b', '<UNK>', text)

    return text

# Aplica no texto já limpo
df['texto_tokens'] = df['texto_limpo'].apply(replace_xxxx_tokens)

# Confere amostra
print(df[['descricao_reclamacao', 'texto_limpo', 'texto_tokens']].head(20))

# Caminho para salvar no Google Drive
path_tokens = BASE_DIR / 'dados_com_tokens.csv'
df.to_csv(path_tokens, index=False, encoding='utf-8')

print(f"\n✅ DataFrame com tokens salvo em: {path_tokens}")


In [ ]:
# 🔧 ETAPA: TOKENIZAÇÃO DOS TEXTOS COM SPACY E SALVAMENTO — COM TQDM

import spacy
from tqdm.notebook import tqdm

# Ativa barra de progresso para operações pandas
tqdm.pandas()

# Carrega modelo SpaCy português
nlp = spacy.load('pt_core_news_sm')

def spacy_tokenizer(text):
    doc = nlp(text)
    return [token.text for token in doc if token.is_alpha]

# Aplica tokenização com barra de progresso
df['texto_tokens_list'] = df['texto_tokens'].progress_apply(spacy_tokenizer)

# Exibe amostra
print(df[['texto_tokens', 'texto_tokens_list']].head(20))

# Salva DataFrame tokenizado
path_tokens_tokenized = BASE_DIR / 'dados_tokens_tokenized.csv'
df.to_csv(path_tokens_tokenized, index=False, encoding='utf-8')

print(f"\n✅ DataFrame com tokens tokenizados via SpaCy salvo em: {path_tokens_tokenized}")


In [ ]:
# 🔧 ETAPA: REMOÇÃO DE STOPWORDS COM LISTA COMBINADA NLTK + SPACY

import nltk
from nltk.corpus import stopwords
import spacy
from tqdm.notebook import tqdm

# Garante download de stopwords NLTK
nltk.download('stopwords')

# Carrega stopwords do NLTK
stopwords_nltk = set(stopwords.words('portuguese'))

# Carrega stopwords do SpaCy
nlp = spacy.load('pt_core_news_sm')
stopwords_spacy = nlp.Defaults.stop_words

# Combina as listas e remove duplicatas
stopwords_combined = stopwords_nltk.union(stopwords_spacy)

print(f"📌 Stopwords NLTK: {len(stopwords_nltk)}")
print(f"📌 Stopwords SpaCy: {len(stopwords_spacy)}")
print(f"📌 Stopwords Combinadas (únicas): {len(stopwords_combined)}")

# Ativa barra de progresso para aplicação
tqdm.pandas()

# Função para filtrar tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stopwords_combined]

# Aplica remoção
df['tokens_sem_stopwords'] = df['texto_tokens_list'].progress_apply(remove_stopwords)

# Verifica amostra
print(df[['texto_tokens_list', 'tokens_sem_stopwords']].head(20))

# Salva DataFrame sem stopwords
path_tokens_no_stopwords = BASE_DIR / 'dados_tokens_no_stopwords.csv'
df.to_csv(path_tokens_no_stopwords, index=False, encoding='utf-8')

print(f"\n✅ DataFrame sem stopwords salvo em: {path_tokens_no_stopwords}")
# 🔧 ETAPA: REMOÇÃO DE STOPWORDS COM LISTA COMBINADA NLTK + SPACY

import nltk
from nltk.corpus import stopwords
import spacy
from tqdm.notebook import tqdm

# Garante download de stopwords NLTK
nltk.download('stopwords')

# Carrega stopwords do NLTK
stopwords_nltk = set(stopwords.words('portuguese'))

# Carrega stopwords do SpaCy
nlp = spacy.load('pt_core_news_sm')
stopwords_spacy = nlp.Defaults.stop_words

# Combina as listas e remove duplicatas
stopwords_combined = stopwords_nltk.union(stopwords_spacy)

print(f"📌 Stopwords NLTK: {len(stopwords_nltk)}")
print(f"📌 Stopwords SpaCy: {len(stopwords_spacy)}")
print(f"📌 Stopwords Combinadas (únicas): {len(stopwords_combined)}")

# Ativa barra de progresso para aplicação
tqdm.pandas()

# Função para filtrar tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stopwords_combined]

# Aplica remoção
df['tokens_sem_stopwords'] = df['texto_tokens_list'].progress_apply(remove_stopwords)

# Verifica amostra
print(df[['texto_tokens_list', 'tokens_sem_stopwords']].head(20))

# Salva DataFrame sem stopwords
path_tokens_no_stopwords = BASE_DIR / 'dados_tokens_no_stopwords.csv'
df.to_csv(path_tokens_no_stopwords, index=False, encoding='utf-8')

print(f"\n✅ DataFrame sem stopwords salvo em: {path_tokens_no_stopwords}")


In [ ]:
# 🔧 ETAPA: LEMATIZAÇÃO COM SPACY E SALVAMENTO

import spacy
from tqdm.notebook import tqdm

# Carrega modelo SpaCy português
nlp = spacy.load('pt_core_news_sm')

# Ativa barra de progresso para lematização
tqdm.pandas()

def lemmatize_tokens(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc if token.is_alpha]

# Aplica lematização
df['tokens_lematizados'] = df['tokens_sem_stopwords'].progress_apply(lemmatize_tokens)

# Exibe amostra
print(df[['tokens_sem_stopwords', 'tokens_lematizados']].head(20))

# Salva DataFrame lematizado
path_tokens_lematizados = BASE_DIR / 'dados_tokens_lematizados.csv'
df.to_csv(path_tokens_lematizados, index=False, encoding='utf-8')

print(f"\n✅ DataFrame com tokens lematizados salvo em: {path_tokens_lematizados}")


# 📊 Macro-Bloco 3 — Vetorização e Engenharia de Features

### 🎯 **Objetivo**
Este bloco foi projetado para criar múltiplas representações vetoriais de cada reclamação da base, usando diferentes abordagens de Processamento de Linguagem Natural (PLN).  
Cada abordagem gera uma matriz de features que alimentará modelos supervisionados de classificação na etapa seguinte.

O propósito é **comparar empiricamente** como cada estratégia de vetorização impacta o desempenho preditivo (com foco em F1 Score weighted ≥ 75%).  
Isso garante decisões fundamentadas sobre qual representação retém melhor a semântica e a estrutura relevante do texto.

---

### 🗂️ **Técnicas implementadas**

A seguir, o pipeline aplica **cinco estratégias de vetorização**, cada uma com sua hipótese de valor para o contexto dos textos de reclamações:

---

## ✅ **A. Bag of Words (BoW)**  
- **Descrição:** Representa o texto por contagem de palavras.  
- **Hipótese:** Palavras isoladas, sem ordem, podem já ser discriminativas para identificar categorias.  
- **Limitação:** Não captura relações entre palavras ou significado contextual.

---

## ✅ **B. TF-IDF (Unigrama)**  
- **Descrição:** Similar ao BoW, mas pondera cada palavra pelo seu peso informativo, penalizando termos muito comuns.  
- **Hipótese:** Palavras raras podem carregar mais valor preditivo.

---

## ✅ **C. TF-IDF (Uni + Bi + Tri-gramas)**  
- **Descrição:** Extende o TF-IDF para considerar pares e trios de palavras consecutivas.  
- **Hipótese:** Expressões compostas e pequenas frases são relevantes para capturar contexto (ex.: “cartão de crédito”, “sem autorização prévia”).

---

## ✅ **D. Word2Vec (CBOW e Skip-Gram)**  
- **Descrição:** Usa embeddings pré-treinados NILC (`cbow_s300.txt` e `skip_s300.txt`), calculando a média vetorial dos tokens lematizados.  
- **Hipótese:** Representa similaridade semântica entre palavras, agrupando contextos similares, mesmo com variações de vocabulário.

---

## ✅ **E. Sentence-Transformer**  
- **Descrição:** Utiliza o modelo `'distiluse-base-multilingual-cased-v2'` para gerar embeddings semânticos de sentenças inteiras.  
- **Hipótese:** Capta relações mais profundas e dependências de longo alcance, superando limitações do Word2Vec ao tratar o texto como uma unidade completa.

---

### ⚙️ **Como será utilizado**
Cada matriz vetorial gerada (A → E) alimentará os mesmos algoritmos de classificação supervisionada (Logistic Regression, Random Forest, etc.).  
Os resultados serão comparados usando métricas padronizadas (Accuracy, Precision, Recall e F1 Score Weighted).  
Esta comparação permitirá selecionar a estratégia de vetorização com maior valor explicativo para os dados, garantindo **robustez metodológica** e **rastreabilidade dos artefatos**.

---


In [ ]:
# 🔧 ETAPA: VETORIZAÇÃO COM COUNTVECTORIZER (UNIGRAMA) + SPLIT

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Junta tokens lematizados em string
df['texto_final'] = df['tokens_lematizados'].apply(lambda tokens: " ".join(tokens))

# Vetoriza com CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(df['texto_final'])

# Vetor alvo
y = df['categoria']

print(f"Shape da matriz vetorial: {X.shape}")

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print(f"Shape X_train: {X_train.shape} | X_test: {X_test.shape}")

# Salva vetorizações como matrizes esparsas se quiser persistir
from scipy import sparse

sparse.save_npz(BASE_DIR / 'X_train_countvec.npz', X_train)
sparse.save_npz(BASE_DIR / 'X_test_countvec.npz', X_test)
y_train.to_csv(BASE_DIR / 'y_train_countvec.csv', index=False)
y_test.to_csv(BASE_DIR / 'y_test_countvec.csv', index=False)

print("\n✅ Vetorização CountVectorizer concluída e salva!")


In [ ]:
# 🔧 ETAPA: VETORIZAÇÃO COM TFIDFVECTORIZER (UNI + BI) + SPLIT

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy import sparse

# Junta tokens lematizados novamente se necessário
df['texto_final'] = df['tokens_lematizados'].apply(lambda tokens: " ".join(tokens))

# Vetoriza com TFIDF (unigrama + bigrama)
vectorizer_tfidf = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer_tfidf.fit_transform(df['texto_final'])

# Vetor alvo
y = df['categoria']

print(f"Shape da matriz vetorial TF-IDF: {X.shape}")

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print(f"Shape X_train: {X_train.shape} | X_test: {X_test.shape}")

# Salva vetorização
sparse.save_npz(BASE_DIR / 'X_train_tfidf_uni_bi.npz', X_train)
sparse.save_npz(BASE_DIR / 'X_test_tfidf_uni_bi.npz', X_test)
y_train.to_csv(BASE_DIR / 'y_train_tfidf_uni_bi.csv', index=False)
y_test.to_csv(BASE_DIR / 'y_test_tfidf_uni_bi.csv', index=False)

print("\n✅ Vetorização TfidfVectorizer (Uni + Bi) concluída e salva!")


In [ ]:
# 🔧 ETAPA: VETORIZAÇÃO COM TFIDFVECTORIZER (UNI + BI + TRI) + SPLIT

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy import sparse

# Junta tokens lematizados se necessário
df['texto_final'] = df['tokens_lematizados'].apply(lambda tokens: " ".join(tokens))

# Vetoriza com TFIDF (uni + bi + tri)
vectorizer_tfidf_tri = TfidfVectorizer(ngram_range=(1,3))
X = vectorizer_tfidf_tri.fit_transform(df['texto_final'])

# Vetor alvo
y = df['categoria']

print(f"Shape da matriz vetorial TF-IDF (Uni + Bi + Tri): {X.shape}")

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print(f"Shape X_train: {X_train.shape} | X_test: {X_test.shape}")

# Salva vetorização
sparse.save_npz(BASE_DIR / 'X_train_tfidf_tri.npz', X_train)
sparse.save_npz(BASE_DIR / 'X_test_tfidf_tri.npz', X_test)
y_train.to_csv(BASE_DIR / 'y_train_tfidf_tri.csv', index=False)
y_test.to_csv(BASE_DIR / 'y_test_tfidf_tri.csv', index=False)

print("\n✅ Vetorização TfidfVectorizer (Uni + Bi + Tri) concluída e salva!")


## Recomendação: faça o download e carregue previamente os arquivos txt cbow_s300 e skip_s300 no diretório de sua utilização antes de continuar

Repositório Original (para ambos CBOW e Skip-gram)

O repositório original para Word Embeddings Pré-treinados em Português é o do NILC, acessível em http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

Repositório do Professor (links diretos para download dos arquivos compactados)

Para o modelo CBOW, o arquivo zip sugerido está em https://dados-ml-pln.s3-sa-east-1.amazonaws.com/cbow_s300.zip.  
Após o download, o arquivo cbow_s300.txt é descompactado para uso

Para o modelo Skip-gram, o arquivo zip sugerido está em https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip.
De forma semelhante, o arquivo skip_s300.txt é extraído para utilização


In [ ]:
# 🔧 PASSO 1: INSTALAÇÃO DAS VERSÕES RECOMENDADAS PARA Word2Vec

!pip install gensim==4.3.2 scipy==1.10.1 numpy==1.23.5 --quiet

print("✅ Dependências fixas instaladas: gensim==4.3.2 | scipy==1.10.1 | numpy==1.23.5")

# 📌 IMPORTANTE: Reinicie o ambiente agora!



In [ ]:
# 🔧 ETAPA: WORD2VEC CBOW 

# 📦 Importações necessárias
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

# 🟢 Ativa barra de progresso para loops demorados
tqdm.pandas()

# 📂 Define BASE_DIR e cria se não existir
BASE_DIR = Path('/content/drive/MyDrive/MBA_NLP/bases_criadas')
BASE_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Diretório BASE_DIR garantido: {BASE_DIR}")

# 📂 Define modelos_dir e valida
modelos_dir = Path('/content/drive/MyDrive/MBA_NLP/modelos')
assert modelos_dir.exists(), f"❌ Diretório {modelos_dir} não existe. Verifique o Drive."

# ✅ Carrega DataFrame se necessário
df_path = BASE_DIR / 'dados_tokens_lematizados.csv'
assert df_path.exists(), f"❌ Arquivo {df_path} não encontrado. Gere-o antes de prosseguir."

df = pd.read_csv(df_path)
print(f"✅ DataFrame carregado de: {df_path}")

# ⚙️ Converte string para lista de tokens se necessário
if isinstance(df['tokens_lematizados'].iloc[0], str):
    df['tokens_lematizados'] = df['tokens_lematizados'].apply(eval)

print(df.head(20))

# ✅ Carrega o modelo CBOW
w2v_cbow_path = modelos_dir / 'cbow_s300.txt'
assert w2v_cbow_path.exists(), f"❌ Modelo CBOW não encontrado em {w2v_cbow_path}"

w2v_cbow = KeyedVectors.load_word2vec_format(str(w2v_cbow_path), binary=False)
w2v_dim = w2v_cbow.vector_size
print(f"✅ Modelo CBOW carregado | Dimensão dos embeddings: {w2v_dim}")

# ⚙️ Função de média vetorial
def get_mean_vector(tokens):
    vectors = [w2v_cbow[word] for word in tokens if word in w2v_cbow]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_dim)

# 🔄 Vetorização com barra de progresso
X_cbow = np.vstack(df['tokens_lematizados'].progress_apply(get_mean_vector))
print(f"✅ Shape da matriz CBOW: {X_cbow.shape}")

# 🎯 Target
y = df['categoria']

# 🔀 Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X_cbow, y, test_size=0.25, stratify=y, random_state=42
)
print(f"✅ Split concluído: X_train {X_train.shape}, X_test {X_test.shape}")

# 💾 Salva vetores e rótulos
np.save(BASE_DIR / 'X_train_word2vec_cbow.npy', X_train)
np.save(BASE_DIR / 'X_test_word2vec_cbow.npy', X_test)
y_train.to_csv(BASE_DIR / 'y_train_word2vec_cbow.csv', index=False)
y_test.to_csv(BASE_DIR / 'y_test_word2vec_cbow.csv', index=False)

print("✅ Vetorização CBOW concluída, artefatos salvos com rastreabilidade.")


In [ ]:
# 🔧 ETAPA: WORD2VEC SKIP-GRAM 

# 📦 Importações
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

tqdm.pandas()

# 📂 BASE_DIR
BASE_DIR = Path('/content/drive/MyDrive/MBA_NLP/bases_criadas')
BASE_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Diretório BASE_DIR garantido: {BASE_DIR}")

# 📂 Modelos
modelos_dir = Path('/content/drive/MyDrive/MBA_NLP/modelos')
assert modelos_dir.exists(), f"❌ Diretório {modelos_dir} não existe. Verifique o Drive."

# ✅ Carrega DataFrame se necessário
df_path = BASE_DIR / 'dados_tokens_lematizados.csv'
assert df_path.exists(), f"❌ Arquivo {df_path} não encontrado. Gere-o antes de prosseguir."

df = pd.read_csv(df_path)
print(f"✅ DataFrame carregado de: {df_path}")

# ⚙️ Converte string para lista se necessário
if isinstance(df['tokens_lematizados'].iloc[0], str):
    df['tokens_lematizados'] = df['tokens_lematizados'].apply(eval)

print(df.head(20))

# ✅ Carrega modelo Skip-Gram
w2v_skip_path = modelos_dir / 'skip_s300.txt'
assert w2v_skip_path.exists(), f"❌ Modelo Skip-Gram não encontrado em {w2v_skip_path}"

w2v_skip = KeyedVectors.load_word2vec_format(str(w2v_skip_path), binary=False)
w2v_dim = w2v_skip.vector_size
print(f"✅ Skip-Gram carregado | Dimensão dos embeddings: {w2v_dim}")

# ⚙️ Função de média vetorial
def get_mean_vector_skip(tokens):
    vectors = [w2v_skip[word] for word in tokens if word in w2v_skip]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v_dim)

# 🔄 Vetorização com barra de progresso
X_skip = np.vstack(df['tokens_lematizados'].progress_apply(get_mean_vector_skip))
print(f"✅ Shape da matriz Skip-Gram: {X_skip.shape}")

# 🎯 Target
y = df['categoria']

# 🔀 Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X_skip, y, test_size=0.25, stratify=y, random_state=42
)
print(f"✅ Split concluído: X_train {X_train.shape}, X_test {X_test.shape}")

# 💾 Salva vetores e rótulos
np.save(BASE_DIR / 'X_train_word2vec_skip.npy', X_train)
np.save(BASE_DIR / 'X_test_word2vec_skip.npy', X_test)
y_train.to_csv(BASE_DIR / 'y_train_word2vec_skip.csv', index=False)
y_test.to_csv(BASE_DIR / 'y_test_word2vec_skip.csv', index=False)

print("✅ Vetorização Skip-Gram concluída, artefatos salvos com rastreabilidade.")


In [ ]:
# 🔧 ETAPA: Sentence Transformers

# REINICIAR O AMBIENTE AGORA!

# ✅ Reinstala compatível para embeddings ST
!pip install -U numpy==1.26.4 sentence-transformers==3.2.1 transformers==4.46.3 --quiet


from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# 📦 Monta Drive
from google.colab import drive
drive.mount('/content/drive')

# 📂 Diretório BASE
BASE_DIR = Path('/content/drive/MyDrive/MBA_NLP/bases_criadas')
print(f"✅ Diretório de bases: {BASE_DIR}")

# ✅ Carrega e normaliza df
df = pd.read_csv(BASE_DIR / 'dados_tokens_lematizados.csv')
df['tokens_lematizados'] = df['tokens_lematizados'].apply(eval)

import unidecode

def normalize_tokens(tokens):
    text = " ".join(tokens)
    text = text.lower()
    text = unidecode.unidecode(text)
    return text

df['texto_final'] = df['tokens_lematizados'].apply(normalize_tokens)
print(df[['tokens_lematizados', 'texto_final']].head(10))

# ✅ Carrega modelo
model_st = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

embeddings = model_st.encode(df['texto_final'].tolist(), show_progress_bar=True)
X = np.array(embeddings)
print(f"✅ Embeddings shape: {X.shape}")

y = df['categoria']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

np.save(BASE_DIR / 'X_train_sentence_transformer.npy', X_train)
np.save(BASE_DIR / 'X_test_sentence_transformer.npy', X_test)
y_train.to_csv(BASE_DIR / 'y_train_sentence_transformer.csv', index=False)
y_test.to_csv(BASE_DIR / 'y_test_sentence_transformer.csv', index=False)

print("✅ Sentence-Transformer embeddings gerados e salvos.")


# 📊 Bloco 4 — Classificação e Avaliação Supervisionada

### 🎯 **Propósito**
Esta etapa compara **todas as estratégias de vetorização** (A–E), geradas no Macro-Bloco 3, usando os **mesmos conjuntos de treino e teste**.  
A meta é avaliar qual técnica gera a melhor performance preditiva na categorização de reclamações, com base em **Accuracy, Precision, Recall e F1 Score Weighted**, visando F1 ≥ 75%.

---

### ⚙️ **Decisões Técnicas**
- **Modelos:** Logistic Regression (baseline) e Random Forest (não linear) para cada abordagem.
- **Carregamento:** Diferencia matrizes esparsas (`CountVectorizer`, `TF-IDF`) e densas (`Word2Vec`, `Sentence-Transformer`).
- **Loop rastreável:** Usa prints claros para indicar formato (`SPARSE` ou `DENSE`), forma da matriz e métricas parciais.
- **Saída:** Relatório `relatorio_comparativo_classificacao.csv` salvo em `/MBA_NLP/bases_criadas` para auditoria e comparações futuras.

---

### 📑 **Resultado esperado**
Ao final, você terá uma tabela padronizada com todos os resultados supervisionados, validando qual feature engineering sustenta melhor performance.


In [ ]:
# 🔧 BLOCO 4 — CLASSIFICAÇÃO & AVALIAÇÃO — PROTOCOLO LLM V5.2

from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import sparse

# 📂 1️⃣ Base de saída garantida
BASE_DIR = Path('/content/drive/MyDrive/MBA_NLP/bases_criadas')
BASE_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Diretório de bases: {BASE_DIR}")

# 📋 2️⃣ Lista de vetores disponíveis
vetores = {
    "bow": ("X_train_countvec.npz", "X_test_countvec.npz", "y_train_countvec.csv", "y_test_countvec.csv"),
    "tfidf_uni": ("X_train_tfidf_uni_bi.npz", "X_test_tfidf_uni_bi.npz", "y_train_tfidf_uni_bi.csv", "y_test_tfidf_uni_bi.csv"),
    "tfidf_tri": ("X_train_tfidf_tri.npz", "X_test_tfidf_tri.npz", "y_train_tfidf_tri.csv", "y_test_tfidf_tri.csv"),
    "word2vec_cbow": ("X_train_word2vec_cbow.npy", "X_test_word2vec_cbow.npy", "y_train_word2vec_cbow.csv", "y_test_word2vec_cbow.csv"),
    "word2vec_skip": ("X_train_word2vec_skip.npy", "X_test_word2vec_skip.npy", "y_train_word2vec_skip.csv", "y_test_word2vec_skip.csv"),
    "sentence_transformer": ("X_train_sentence_transformer.npy", "X_test_sentence_transformer.npy", "y_train_sentence_transformer.csv", "y_test_sentence_transformer.csv"),
}

# 📊 3️⃣ DataFrame de resultados
resultados = []

# 🔁 4️⃣ Loop para cada abordagem
for name, (X_train_file, X_test_file, y_train_file, y_test_file) in vetores.items():

    # ⚙️ Carrega vetores SPARSE ou DENSE
    if "countvec" in X_train_file or "tfidf" in X_train_file:
        X_train = sparse.load_npz(BASE_DIR / X_train_file)
        X_test = sparse.load_npz(BASE_DIR / X_test_file)
        print(f"✅ {name.upper()} carregado como SPARSE: {X_train.shape}")
    else:
        X_train = np.load(BASE_DIR / X_train_file)
        X_test = np.load(BASE_DIR / X_test_file)
        print(f"✅ {name.upper()} carregado como DENSE: {X_train.shape}")

    # 🎯 Target labels
    y_train = pd.read_csv(BASE_DIR / y_train_file).squeeze()
    y_test = pd.read_csv(BASE_DIR / y_test_file).squeeze()

    # 🚀 Modelos supervisonados
    models = {
        "LogisticRegression": LogisticRegression(max_iter=1000),
        "RandomForest": RandomForestClassifier(n_estimators=100)
    }

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        resultados.append({
            "Vetorizacao": name,
            "Modelo": model_name,
            "Accuracy": round(acc, 4),
            "Precision": round(prec, 4),
            "Recall": round(rec, 4),
            "F1_Score_Weighted": round(f1, 4)
        })

        print(f"✅ [{name.upper()}] {model_name} | F1 Score Weighted: {f1:.4f}")

# 💾 5️⃣ Relatório final
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv(BASE_DIR / "relatorio_comparativo_classificacao.csv", index=False)
print(f"\n✅ Relatório comparativo salvo em: {BASE_DIR / 'relatorio_comparativo_classificacao.csv'}")

df_resultados


# 📊 Relatório Comparativo — Vetorização & Classificação

✅ **Diretório de bases:** `/content/drive/MyDrive/MBA_NLP/bases_criadas`

| Abordagem                    | Shape             | F1 Score Weighted |
|------------------------------|-------------------|-------------------|
| **BoW** LogisticRegression   | SPARSE (15804, 36496) | **0.9005** |
| BoW RandomForest             | SPARSE (15804, 36496) | 0.8048 |
| TFIDF_UNI LogisticRegression | SPARSE (15804, 779791) | 0.8931 |
| TFIDF_UNI RandomForest       | SPARSE (15804, 779791) | 0.8085 |
| TFIDF_TRI LogisticRegression | SPARSE (15804, 2,552,999) | 0.8787 |
| TFIDF_TRI RandomForest       | SPARSE (15804, 2,552,999) | 0.7794 |
| Word2Vec CBOW LogisticRegression | DENSE (15804, 300) | 0.8007 |
| Word2Vec CBOW RandomForest   | DENSE (15804, 300) | 0.7119 |
| Word2Vec Skip LogisticRegression | DENSE (15804, 300) | 0.8121 |
| Word2Vec Skip RandomForest   | DENSE (15804, 300) | 0.7278 |
| Sentence-Transformer LogisticRegression | DENSE (14750, 512) | 0.7722 |
| Sentence-Transformer RandomForest     | DENSE (14750, 512) | 0.7098 |

✅ **Relatório comparativo salvo em:**  
`/content/drive/MyDrive/MBA_NLP/bases_criadas/relatorio_comparativo_classificacao.csv`

---

### 📌 **Resumo**

- O baseline escolhido é **BoW + LogisticRegression**, com **F1 Weighted de 0.9005**, superando a meta de 0.75.
- Todas as demais abordagens ficam documentadas para auditoria, comparação futura e experimentação avançada.

---

---
---
---


###**Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

**Atenção:**
- **Implemente aqui apenas os scripts que fazem parte do modelo campeão.**
- **Execute o pipeline do modelo campeão completamente para garantir que não tetá erros no script.**


---
---
O pipeline completo do modelo campeão, que usa Bag of Words (BoW), foi reescrito com o objetivo de consolidar tudo em blocos autocontidos, organizados e executáveis em qualquer ambiente.
---
---

## Etapa 0 — Setup de Dependências e Bibliotecas

Antes de executar o pipeline do *modelo campeão*, garantimos que todas as bibliotecas e recursos estejam disponíveis.  
Este bloco prepara o ambiente, faz downloads necessários (`nltk` e `SpaCy`), carrega o modelo de linguagem em português e combina as listas de stopwords `nltk` + `SpaCy`.



In [2]:
# 🔧 ETAPA: SETUP DE DEPENDÊNCIAS E BIBLIOTECAS

# Instalar pandas se necessário
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

# Instalar numpy se necessário
try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

# Módulos padrão da biblioteca padrão Python (não precisam de instalação)
import string
import re
import os

# Instalar unidecode se necessário
try:
    import unidecode
except ImportError:
    !pip install unidecode
    import unidecode

# Instalar nltk se necessário
try:
    import nltk
except ImportError:
    !pip install nltk
    import nltk

from nltk.corpus import stopwords

# Instalar spacy se necessário
try:
    import spacy
except ImportError:
    !pip install -U spacy
    import spacy

# Instalar sklearn se necessário
try:
    import sklearn
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, f1_score, confusion_matrix
except ImportError:
    !pip install scikit-learn
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, f1_score, confusion_matrix

# Instalar tqdm se necessário
try:
    from tqdm.notebook import tqdm
except ImportError:
    !pip install tqdm
    from tqdm.notebook import tqdm

# Instalar IPython.display se necessário (normalmente vem com Jupyter)
try:
    from IPython.display import display
except ImportError:
    !pip install IPython
    from IPython.display import display

# Baixar stopwords PT do nltk
nltk.download('stopwords')

# Baixar e carregar SpaCy pt_core_news_sm
try:
    nlp = spacy.load('pt_core_news_sm')
except:
    import subprocess
    subprocess.run(["python", "-m", "spacy", "download", "pt_core_news_sm"])
    nlp = spacy.load('pt_core_news_sm')

# Combinar stopwords NLTK + SpaCy
stopwords_pt = set(stopwords.words('portuguese'))
stopwords_spacy = nlp.Defaults.stop_words
combined_stopwords = stopwords_pt.union(stopwords_spacy)

print(f"✅ Todas as dependências foram verificadas e carregadas.")
print(f"Stopwords combinadas: {len(combined_stopwords)} termos")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 73.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 90.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.3/869.3 kB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip inst

[nltk_data] Downloading package stopwords to /home/wrm/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 65.5 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


✅ Todas as dependências foram verificadas e carregadas.
Stopwords combinadas: 500 termos


## Etapa 1 — Pipeline do Modelo Campeão com Lematização, Placeholders e Vetorização BoW

Este bloco executa o pipeline do *modelo campeão* de forma **portável** e **rastreável**, incluindo:
- **Carregamento do dataset** pela URL oficial,
- **Pré-processamento** com substituição inteligente de placeholders (`<DATE>`, `<NUMBER>`, `<PII>`),
- **Tokenização e lematização** com SpaCy,
- **Remoção de stopwords combinadas** (`nltk` + `SpaCy`),
- **Barra de progresso `tqdm`** para monitorar o avanço da lematização,
- **Montagem do texto final** para vetorização,
- Vetorização com **CountVectorizer (unigrama)**,
- **Divisão treino/teste** estratificada com `random_state=42`,
- Treinamento do **LogisticRegression**,
- Avaliação com **classification_report**, **F1 Score weighted** e matriz de confusão.



In [3]:
# 🔧 ETAPA: PIPELINE COMPLETO DO MODELO **CAMPEÃO** 
# 1️⃣ Carregar dataset
url = "https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv"
df = pd.read_csv(url, sep=';')
print(df.head(5))

# 2️⃣ Limpeza básica
df.dropna(subset=['descricao_reclamacao'], inplace=True)

# 3️⃣ Substituição inteligente de placeholders
def replace_placeholders(text):
    text = re.sub(r'\b\d{2}/\d{2}/\d{4}\b', '<DATE>', text)
    text = re.sub(r'\b\d{2}-\d{2}-\d{4}\b', '<DATE>', text)
    text = re.sub(r'\b\d{4}\b', '<YEAR>', text)
    text = re.sub(r'\b\d+\b', '<NUMBER>', text)
    text = re.sub(r'X{2,}', '<PII>', text, flags=re.IGNORECASE)
    return text

# 4️⃣ Função de pré-processamento + lematização
def preprocess_and_lemmatize(text):
    text = replace_placeholders(text)
    text = text.lower()
    text = unidecode.unidecode(text)
    text = re.sub(r'\s+', ' ', text).strip()
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc 
        if token.is_alpha and token.lemma_ not in combined_stopwords
    ]
    return tokens

# 5️⃣ Usar tqdm para progresso
from tqdm.notebook import tqdm
tqdm.pandas()

df['tokens_lematizados'] = df['descricao_reclamacao'].progress_apply(preprocess_and_lemmatize)

print(df[['descricao_reclamacao', 'tokens_lematizados']].head(5))

# 6️⃣ Texto final para vetorização
df['texto_final'] = df['tokens_lematizados'].apply(lambda tokens: " ".join(tokens))

# 7️⃣ Vetorização BoW
vectorizer = CountVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(df['texto_final'])
y = df['categoria']

print(f"Shape matriz vetorial: {X.shape}")

# 8️⃣ Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")

# 9️⃣ Treinar modelo
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# 🔟 Avaliar com apresentação formatada
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import pandas as pd
from IPython.display import display

y_pred = clf.predict(X_test)

# Report
report = classification_report(y_test, y_pred, target_names=clf.classes_, digits=2)
print("\n🔎 **Classification Report**:\n")
print(report)

# F1 Score weighted
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"\n✅ **F1 Score (weighted): {f1:.2%}**")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
labels = clf.classes_
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

print("\n🔍 **Matriz de Confusão:**")
display(cm_df)


   id_reclamacao              data_abertura  \
0        3229299  2019-05-01T12:00:00-05:00   
1        3199379  2019-04-02T12:00:00-05:00   
2        3233499  2019-05-06T12:00:00-05:00   
3        3180294  2019-03-14T12:00:00-05:00   
4        3224980  2019-04-27T12:00:00-05:00   

                             categoria  \
0              Hipotecas / Empréstimos   
1  Cartão de crédito / Cartão pré-pago   
2  Cartão de crédito / Cartão pré-pago   
3  Cartão de crédito / Cartão pré-pago   
4           Serviços de conta bancária   

                                descricao_reclamacao  
0  Bom dia, meu nome é xxxx xxxx e agradeço se vo...  
1  Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...  
2  O cartão Chase foi relatado em xx/xx/2019. No ...  
3  Em xx/xx/2018, enquanto tentava reservar um ti...  
4  Meu neto me dê cheque por {$ 1600,00} Eu depos...  


  0%|          | 0/21072 [00:00<?, ?it/s]

                                descricao_reclamacao  \
0  Bom dia, meu nome é xxxx xxxx e agradeço se vo...   
1  Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...   
2  O cartão Chase foi relatado em xx/xx/2019. No ...   
3  Em xx/xx/2018, enquanto tentava reservar um ti...   
4  Meu neto me dê cheque por {$ 1600,00} Eu depos...   

                                  tokens_lematizados  
0  [dia, nome, pii, pii, agradeco, voce, puder, a...  
1  [atualizei, cartao, pii, pii, informar, por o,...  
2  [cartao, chase, relatar, em o, entanto, pedido...  
3  [reservar, ticket, pii, pii, deparar, oferta, ...  
4  [neto, cheque, depositei, em o, conta, chase, ...  
Shape matriz vetorial: (21072, 29483)
X_train: (15804, 29483), X_test: (5268, 29483)

🔎 **Classification Report**:

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.91      0.92      0.91      1252
            Hipotecas / Empréstimos       0.91      0.91      

,Cartão de crédito / Cartão pré-pago,Hipotecas / Empréstimos,Outros,Roubo / Relatório de disputa,Serviços de conta bancária
Cartão de crédito / Cartão pré-pago,1149,20,21,48,14
Hipotecas / Empréstimos,16,880,14,29,23
Outros,22,23,469,17,27
Roubo / Relatório de disputa,46,30,19,1058,53
Serviços de conta bancária,28,15,15,51,1181
